This example simulates the variation of a tally responce as a function of time. This particular tally is the tritium breeding ratio and this tends to decrease over time as the lithium gets burnt up by neutron irradiation

In [ ]:
import openmc
import openmc.deplete
import matplotlib.pyplot as plt
import math

lithium_orthosilicate_radius = 250

# MATERIALS

mats = openmc.Materials()

# makes a simple material from Iron
breeding_material = openmc.Material(name="breeding_material") 
breeding_material.add_elements_from_formula('Li4SiO4')
breeding_material.set_density('g/cm3', 2.5)
breeding_material.volume = (4/3) * math.pi * lithium_orthosilicate_radius**3
breeding_material.depletable = True

materials = openmc.Materials([breeding_material])
materials.export_to_xml()


# GEOMETRY

# surfaces
sph1 = openmc.Sphere(r=lithium_orthosilicate_radius, boundary_type='vacuum')

# cells
shield_cell = openmc.Cell(region=-sph1)
shield_cell.fill = breeding_material
shield_cell.volume = (4/3) * math.pi * sph1.r**3

universe = openmc.Universe(cells=[shield_cell])
geometry = openmc.Geometry(universe)

This section defines the neutron source term to use and the settings

In [ ]:
# creates a 14MeV neutron point source
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
source.particles = 'neutron'

# SETTINGS

# Instantiate a Settings object
settings = openmc.Settings()
settings.batches = 2
settings.inactive = 0
settings.particles = 500
settings.source = source
settings.run_mode = 'fixed source'


tallies = openmc.Tallies()

# added a cell tally for tritium production
cell_filter = openmc.CellFilter(shield_cell)
tbr_tally = openmc.Tally(name='TBR')
tbr_tally.filters = [cell_filter]
tbr_tally.scores = ['(n,Xt)']  # Where X is a wildcard character, this catches any tritium production
tallies.append(tbr_tally)

geometry.export_to_xml()
settings.export_to_xml()
tallies.export_to_xml()
materials.export_to_xml()
model = openmc.model.Model(geometry, materials, settings, tallies)

This is the depltion specific part of the model setup.
Here we:

    specify the chain file, this tells openmc the decay paths between isotopes including proabilities of different routes and half lives
    
    set the time steps and corresponding source rates 

In [ ]:
# This chain_endfb71 file was made with the run python generate_endf71_chain.py from the openmc-dev/data repo
chain_filename = 'chain_endfb71.xml'
chain = openmc.deplete.Chain.from_xml(chain_filename)

operator = openmc.deplete.Operator(model, chain_filename)

time_steps = [365*24*60*60] * 5 # 5 steps of 5 years in seconds
source_rates = [1e9]*5 # 1GW

integrator = openmc.deplete.PredictorIntegrator(operator, time_steps, source_rates)

integrator.integrate()

Access the TBR tally at each depletion time step

In [ ]:
for counter in [0,1,2,3,4,5]:
    sp = openmc.StatePoint(f'openmc_simulation_n{counter}.h5')
    tbr_tally = sp.get_tally(name='TBR')
    print(f'depletion step {counter} TBR={tbr_tally.mean.sum()}')

Certain reations are also available in the depletion results file. 

For example you can access the rate of n,gamma reactions at each time step

In [ ]:
results = openmc.deplete.ResultsList.from_hdf5("depletion_results.h5")

times, number_of_n_gamma_reactions = results.get_reaction_rate(breeding_material, 'Li6', '(n,gamma)')
number_of_n_gamma_reactions